In [17]:
# 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt

#성능평가 모듈
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

#데이터 준비

data = pd.read_csv('../datas/train_V2.csv')
#sample = pd.read_csv('../datas/sample_submission_V2.csv')
#컬럼명 확인하고 필요 없는 거 날리기
data


,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.000,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.040,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.000,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.000,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.000,0,0,49.75,2,0,0.1875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446961,afff7f652dbc10,d238e426f50de7,18492834ce5635,0,0,0.00,0,0,0,74,...,0,1292.0000,0,0.000,0,0,1019.00,3,1507,0.1786
4446962,f4197cf374e6c0,408cdb5c46b2ac,ee854b837376d9,0,1,44.15,0,0,0,69,...,0,0.0000,0,0.000,0,0,81.70,6,0,0.2935
4446963,e1948b1295c88a,e26ac84bdf7cef,6d0cd12784f1ab,0,0,59.06,0,0,0,66,...,0,0.0000,0,2.184,0,0,788.70,4,0,0.4815
4446964,cc032cdd73b7ac,c2223f35411394,c9c701d0ad758a,0,4,180.40,1,1,2,11,...,2,0.0000,0,0.000,0,0,2748.00,8,0,0.8000


In [18]:
#winpoints와 rankpoints: 버전만 다른 같은 데이터라 합쳐버림
newcol = data[['winPoints', 'rankPoints']]
winrank = newcol.sum(axis = 1)


In [19]:
usingdata = data.drop(['Id', 'groupId', 'matchId','maxPlace', 'killPoints', 'kills', 'rankPoints', 'rideDistance', 'roadKills', 'swimDistance', 'vehicleDestroys', 'winPoints'], axis=1)
usingdata['winRank'] = winrank

usingdata.columns
usingdata = usingdata[['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPlace', 'killStreaks', 'longestKill', 'matchDuration', 'matchType',
       'numGroups', 'revives', 'teamKills', 'walkDistance', 'weaponsAcquired',
       'winRank','winPlacePerc', ]]

In [20]:
# 게임별로 쪼개기
#normal_solo_fpp 
mask1 = usingdata['matchType'].isin(['normal-solo-fpp'])
nsolo_fpp = usingdata[mask1].drop(['matchType'], axis=1)
nsolo_fpp = nsolo_fpp.reset_index(drop=True)

#normal_duo_fpp
mask2 = usingdata['matchType'].isin(['normal-duo-fpp'])
nduo_fpp = usingdata[mask2].drop(['matchType'], axis=1)
nduo_fpp = nduo_fpp.reset_index(drop=True)

#normal_squad_fpp
mask3 = usingdata['matchType'].isin(['normal-squad-fpp'])
nsquad_fpp = usingdata[mask3].drop(['matchType'], axis=1)
nsquad_fpp = nsquad_fpp.reset_index(drop=True)

#normal_solo
mask4 = usingdata['matchType'].isin(['normal-solo'])
nsolo = usingdata[mask4].drop(['matchType'], axis=1)
nsolo = nsolo.reset_index(drop=True)

#normal_duo
mask5 = usingdata['matchType'].isin(['normal-duo'])
nduo = usingdata[mask5].drop(['matchType'], axis = 1)
nduo = nduo.reset_index(drop=True)

#normal_squad
mask6 = usingdata['matchType'].isin(['normal-squad'])
nsquad = usingdata[mask6].drop(['matchType'], axis = 1)
nsquad = nsquad.reset_index(drop=True)

In [21]:
nsolo_fpp

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killStreaks,longestKill,matchDuration,numGroups,revives,teamKills,walkDistance,weaponsAcquired,winRank,winPlacePerc
0,2,0,655.20,0,2,0,11,3,1.480,492,13,0,0,29.570,9,1499,0.4737
1,0,2,85.67,0,0,7,3,1,4.511,314,10,0,0,187.100,1,1500,0.7778
2,0,3,364.40,0,2,0,1,1,101.800,993,13,0,0,2468.000,2,1500,0.9167
3,1,0,256.20,0,1,0,16,1,51.360,1084,16,0,0,8.202,2,1500,0.7826
4,2,0,1750.00,0,4,5,3,6,37.080,898,15,0,0,0.000,13,1500,0.8947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,4,0,1634.00,0,2,0,7,2,52.120,895,16,0,0,1026.000,34,1500,0.6842
1678,6,0,3297.00,0,5,4,1,5,203.400,896,13,0,0,253.000,32,1500,1.0000
1679,3,0,548.00,0,1,2,20,1,198.900,893,12,0,0,221.900,20,1500,0.4783
1680,0,0,41.95,0,0,0,48,1,43.380,599,27,0,0,0.000,4,1500,0.9434


In [22]:
#nsolofpp
target = nsolo_fpp[nsolo_fpp.columns[-1]]
feature = nsolo_fpp[nsolo_fpp.columns[:-1]]

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
xtrain,xtest,ytrain,ytest = train_test_split(feature, target, test_size=0.3, random_state=42)

In [25]:
print(f'[TRAIN SHAPE]: {xtrain.shape},{ytrain.shape}')
print(f'[TEST SHAPE]: {xtest.shape},{ytest.shape}')

[TRAIN SHAPE]: (1177, 16),(1177,)
[TEST SHAPE]: (505, 16),(505,)


In [26]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler1.fit(xtrain)
scaler1.fit(xtest)

xtrain = scaler1.transform(xtrain)
xtest = scaler1.transform(xtest)

In [27]:
from sklearn.ensemble import RandomForestRegressor

reg1 = RandomForestRegressor()
reg1.fit(xtrain,ytrain)
ypred_nsolo_fpp = reg1.predict(xtest)
nsf_mse = mean_squared_error(ytest, ypred_nsolo_fpp)
nsf_mae = mean_absolute_error(ytest, ypred_nsolo_fpp)
nsf_r2 = r2_score(ytest, ypred_nsolo_fpp)
nsf_rmse = mean_squared_error(ytest, ypred_nsolo_fpp)

In [28]:
nsolofpp_testscore = reg1.score(xtest,ytest)
nsolofpp_trainscore = reg1.score(xtrain,ytrain)

In [29]:
#nduo_fpp
target = nsolo[nduo_fpp.columns[-1]]
feature = nsolo[nduo_fpp.columns[:-1]]

xtrain,xtest,ytrain,ytest = train_test_split(feature, target, test_size=0.3, random_state=42)




In [30]:
scaler2 = StandardScaler()
scaler2.fit(xtrain)
scaler2.fit(xtest)

xtrain = scaler2.transform(xtrain)
xtest = scaler2.transform(xtest)


In [31]:
from sklearn.ensemble import RandomForestRegressor

reg2 = RandomForestRegressor()
reg2.fit(xtrain,ytrain)

RandomForestRegressor()

In [32]:
nduofpp_testscore = reg2.score(xtest,ytest)
nduofpp_trainscore = reg2.score(xtrain,ytrain)
ypred_nduo_fpp = reg2.predict(xtest)
ndf_mse = mean_squared_error(ytest, ypred_nduo_fpp)
ndf_mae = mean_absolute_error(ytest, ypred_nduo_fpp)
ndf_r2 = r2_score(ytest, ypred_nduo_fpp)
ndf_rmse = mean_squared_error(ytest, ypred_nduo_fpp)

In [33]:
#normal_squad_fpp

target = nsquad_fpp[nsquad_fpp.columns[-1]]
feature = nsquad_fpp[nsquad_fpp.columns[:-1]]

In [34]:
xtrain, xtest, ytrain, ytest = train_test_split(feature, target)

scaler3 = StandardScaler()
scaler3.fit(xtrain)
scaler3.fit(xtest)

xtrain = scaler3.transform(xtrain)
xtest = scaler3.transform(xtest)

reg3 = RandomForestRegressor()
reg3.fit(xtrain,ytrain)
ypred_nsquad_fpp = reg3.predict(xtest)

nsqf_mse = mean_squared_error(ytest, ypred_nsquad_fpp)
nsqf_mae = mean_absolute_error(ytest, ypred_nsquad_fpp)
nsqf_r2 = r2_score(ytest, ypred_nsquad_fpp)
nsqf_rmse = mean_squared_error(ytest, ypred_nsquad_fpp)
nsquadfpp_testscore = reg3.score(xtest,ytest)
nsquadfpp_trainscore = reg3.score(xtrain,ytrain)

In [35]:
#normal_solo

nstarget = nsolo[nsolo.columns[-1]]
nsfeature = nsolo[nsolo.columns[:-1]]

In [36]:
xtrain, xtest, ytrain, ytest = train_test_split(feature, target)

scaler4 = StandardScaler()
scaler4.fit(xtrain)
scaler4.fit(xtest)

xtrain = scaler4.transform(xtrain)
xtest = scaler4.transform(xtest)

reg4 = RandomForestRegressor()
reg4.fit(xtrain,ytrain)
ypred_nsolo = reg4.predict(xtest)


ns_mse = mean_squared_error(ytest, ypred_nsolo)
ns_mae = mean_absolute_error(ytest, ypred_nsolo)
ns_r2 = r2_score(ytest, ypred_nsolo)
ns_rmse = mean_squared_error(ytest, ypred_nsolo)

nsolo_testscore = reg4.score(xtest,ytest)
nsolo_trainscore = reg4.score(xtrain,ytrain)

In [37]:
#normal_duo

ndtarget = nduo[nduo.columns[-1]]
ndfeature = nduo[nduo.columns[:-1]]

xtrain, xtest, ytrain, ytest = train_test_split(feature, target)

scaler5 = StandardScaler()
scaler5.fit(xtrain)
scaler5.fit(xtest)

xtrain = scaler5.transform(xtrain)
xtest = scaler5.transform(xtest)

reg5 = RandomForestRegressor()
reg5.fit(xtrain,ytrain)

ypred_nduo = reg5.predict(xtest)

nd_mse = mean_squared_error(ytest, ypred_nduo)
nd_mae = mean_absolute_error(ytest, ypred_nduo)
nd_r2 = r2_score(ytest, ypred_nduo)
nd_rmse = mean_squared_error(ytest, ypred_nduo)

nduo_testscore = reg5.score(xtest,ytest)
nduo_trainscore = reg5.score(xtrain,ytrain)

In [38]:
#normal squad

nsqtarget = nsquad[nsquad.columns[-1]]
nsqfeature = nsquad[nsquad.columns[:-1]]

xtrain, xtest, ytrain, ytest = train_test_split(feature, target)

scaler6 = StandardScaler()
scaler6.fit(xtrain)
scaler6.fit(xtest)

xtrain = scaler6.transform(xtrain)
xtest = scaler6.transform(xtest)

reg6 = RandomForestRegressor()
reg6.fit(xtrain,ytrain)
ypred_nsquad = reg6.predict(xtest)

nsq_mse = mean_squared_error(ytest, ypred_nsquad)
nsq_mae = mean_absolute_error(ytest, ypred_nsquad)
nsq_r2 = r2_score(ytest, ypred_nsquad)
nsq_rmse = mean_squared_error(ytest, ypred_nsquad)

nsquad_testscore = reg6.score(xtest,ytest)
nsquad_trainscore = reg6.score(xtrain,ytrain)

In [39]:
print(f'''
[Normal Solo FPP]
- Trainscore : {nsolofpp_trainscore}
- Testscore : {nsolofpp_testscore}

[Normal Duo FPP]
-Trainscore : {nduofpp_trainscore}
-Testscore : {nduofpp_testscore}

[Normal Squad FPP]
- Trainscore : {nsquadfpp_trainscore}
- Testscore : {nsquadfpp_testscore}

[Normal Solo]
- Trainscore : {nsolo_trainscore}
- Testscore : {nsolo_testscore}

[Normal Duo]
- Trainscore : {nduo_trainscore}
- Testscore : {nduo_testscore}

[Normal Squad]
- Trainscore : {nsquad_trainscore}
- Testscore : {nsquad_testscore}
''')




[Normal Solo FPP]
- Trainscore : 0.9487047834611334
- Testscore : 0.5555805440492463

[Normal Duo FPP]
-Trainscore : 0.9373119648903021
-Testscore : 0.5159929103395284

[Normal Squad FPP]
- Trainscore : 0.9326491015890009
- Testscore : 0.5334892763551733

[Normal Solo]
- Trainscore : 0.9327391498069777
- Testscore : 0.5278912811591052

[Normal Duo]
- Trainscore : 0.9332613231951306
- Testscore : 0.5012671538106448

[Normal Squad]
- Trainscore : 0.9326821366161244
- Testscore : 0.5157069268740183


In [40]:
#Normal Solo First Person Perspective:  삭제(사유 : 다른 것보다 유희의 성격이 강한 맵이라 다른 분류들과 상관계수가 다름)0
#점수가 왜 -5점대지..?
#아니다. 오타엿다. 잘못 잡았다!
#그래서 수정햇다

In [41]:
# 성능 평가는 변수 겹침 이슈로 위에 다 적었다
# 프린트만 한번에 합시다

print(f'''[Performance] 
[Normal Solo FPP]
-MSE : {nsf_mse}
-MAE : {nsf_mae}
-RMSE : {nsf_rmse}
-R2 : {nsf_r2}

[Normal Duo FPP]
-MSE : {ndf_mse}
-MAE : {ndf_mae}
-RMSE : {ndf_rmse}
-R2 : {ndf_r2}

[Normal Squad FPP]
-MSE : {nsqf_mse}
-MAE : {nsqf_mae}
-RMSE : {nsqf_rmse}
-R2 : {nsqf_r2}

[Normal Solo]
-MSE : {ns_mse}
-MAE : {ns_mae}
-RMSE : {ns_rmse}
-R2 : {ns_r2}

[Normal Duo]
-MSE : {nd_mse}
-MAE : {nd_mae}
-RMSE : {nd_rmse}
-R2 : {nd_r2}

[Normal Squad]
-MSE : {nsq_mse}
-MAE : {nsq_mae}
-RMSE : {nsq_rmse}
-R2 : {nsq_r2}
''')

[Performance] 
[Normal Solo FPP]
-MSE : 0.044166915224839606
-MAE : 0.1542093702970297
-RMSE : 0.044166915224839606
-R2 : 0.5555805440492463

[Normal Duo FPP]
-MSE : 0.045851241519591836
-MAE : 0.15877281632653056
-RMSE : 0.045851241519591836
-R2 : 0.5159929103395284

[Normal Squad FPP]
-MSE : 0.04906771267625315
-MAE : 0.1663395027945971
-RMSE : 0.04906771267625315
-R2 : 0.5334892763551733

[Normal Solo]
-MSE : 0.049519246056529576
-MAE : 0.16656899953423382
-RMSE : 0.049519246056529576
-R2 : 0.5278912811591052

[Normal Duo]
-MSE : 0.05374525824196111
-MAE : 0.1742361371681416
-RMSE : 0.05374525824196111
-R2 : 0.5012671538106448

[Normal Squad]
-MSE : 0.05112133759651397
-MAE : 0.16924720889613415
-RMSE : 0.05112133759651397
-R2 : 0.5157069268740183


In [42]:
# 모델 저장
import joblib
import os

In [51]:
md_filename = 'pubg_normal'
md_dir = '../model/'

# 저장폴더 존재여부 확인 후 저장하기
if not os.path.exists(md_dir):
    os.mkdir(md_dir)
    
md_filename2 = 'pubg_normal_Scaler'


In [52]:
joblib.dump(scaler1, md_dir+md_filename2+'scaler1'+'.pkl')
joblib.dump(scaler2, md_dir+md_filename2+'scaler2'+'.pkl')
joblib.dump(scaler3, md_dir+md_filename2+'scaler3'+'.pkl')
joblib.dump(scaler4, md_dir+md_filename2+'scaler4'+'.pkl')
joblib.dump(scaler5, md_dir+md_filename2+'scaler5'+'.pkl')
joblib.dump(scaler6, md_dir+md_filename2+'scaler6'+'.pkl')



['../model/pubg_normal_Scalerscaler6.pkl']

In [53]:

joblib.dump(reg1,md_dir + md_filename +'_solo_fpp' + '.pkl')


['../model/pubg_normal_solo_fpp.pkl']

In [54]:
joblib.dump(reg2,md_dir + md_filename +'_duo_fpp'+ '.pkl')

['../model/pubg_normal_duo_fpp.pkl']

In [55]:
joblib.dump(reg3,md_dir + md_filename + '_squad_fpp'+ '.pkl')

['../model/pubg_normal_squad_fpp.pkl']

In [56]:
joblib.dump(reg4,md_dir + md_filename +'_solo'+ '.pkl')

['../model/pubg_normal_solo.pkl']

In [57]:
joblib.dump(reg5,md_dir + md_filename+'_duo'+ '.pkl')

['../model/pubg_normal_duo.pkl']

In [50]:
joblib.dump(reg6,md_dir + md_filename +'_squad'+ '.pkl')

['../model/pubg_normal_squad.pkl']